# 1. スクレイピング

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install langchain

In [ ]:
!pip install trafilatura youtube_transcript_api

## 徳島大学のWebページ
trafilaturaを使う



In [ ]:
from trafilatura import fetch_url, extract

web_urls = [
    "https://ja.wikipedia.org/wiki/%E5%BE%B3%E5%B3%B6%E5%A4%A7%E5%AD%A6",
    "https://www.tokushima-u.ac.jp/admission/seikatsu/life.html",
]

web_docs = ""
for url in web_urls:
    doc = fetch_url(url)
    web_docs += extract(doc)
print(web_docs[:100])

### 補足：Wikipedia APIの利用

In [ ]:
import requests

# 検索するキーワード
search_keyword = "徳島大学"

# API URL
api_url = "https://ja.wikipedia.org/w/api.php"

# API定義
params = {
    "action": "query",
    "format": "json",
    "titles": search_keyword,
    "prop": "extracts",
    "explaintext": True,
}

response = requests.get(api_url, params=params)
data = response.json()

# ページの内容を抽出
page_id = list(data["query"]["pages"].keys())[0]  # ページのユニーク識別子を取得
page_content = data["query"]["pages"][page_id].get("extract", "no doc")

print(page_content)

## 徳島大学のYouTubeの字幕

In [ ]:
from langchain.document_loaders import YoutubeLoader

# 徳島大学のYouTube動画
youtube_urls = [
    "https://youtu.be/IKAA3rKNvNQ?si=ClOsDZCX50UHVmZM",
    "https://youtu.be/AyZm8iuDGgM?si=BTUF_wYzt44jxjDB",
]

youtube_docs = ""
for url in youtube_urls:
    loader = YoutubeLoader.from_youtube_url(youtube_url=url, language="ja")
    docs = loader.load()
    if docs:
        youtube_docs += docs[0].page_content
    else:
        print("no docs")
print(youtube_docs[:100])

### 補足：YouTube APIの利用


In [ ]:
!pip install google-api-python-client

In [ ]:
import os
from googleapiclient.discovery import build
from langchain.document_loaders import YoutubeLoader
import csv

# 開発者キーを設定する
api_key = ""
# あなたの開発者キーに置き換えてください

youtube = build("youtube", "v3", developerKey=api_key)

# "徳島大学" キーワードを含むビデオの URL を保存するための変数です。
youtube_urls = []

# 検索
search_response = (
    youtube.search()
    .list(
        q="徳島大学",  # キーワー
        type="video",
        part="id",
        maxResults=10,  # 数量
        regionCode="JP",  # 地域コードを日本（JP）に指定
    )
    .execute()
)

# 検索結果の各ビデオを走査し、ビデオのURLを取得します"。
for search_result in search_response.get("items", []):
    video_id = search_result["id"]["videoId"]
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    youtube_urls.append(video_url)

# 字幕情報を格納するリスト
captions_info = []
no_docs_count = 0

for url in youtube_urls:
    loader = YoutubeLoader.from_youtube_url(youtube_url=url, language="ja")
    docs = loader.load()
    if docs:
        # 字幕テキストを取得
        youtube_text = docs[0].page_content
        # 字幕情報をリストに追加"
        captions_info.append({"url": url, "text": youtube_text})
    else:
        print(f"No docs for URL: {url}")
        no_docs_count += 1

# 字幕情報を TSV ファイルに保存
tsv_file = "youtube_text.tsv"
with open(tsv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter="\t", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #
    writer.writerow(["URL", "Text"])
    for caption in captions_info:
        writer.writerow([caption["url"], caption["text"]])
print(f"{tsv_file} ファイルに字幕情報が保存されました。")
print(f"取得データ情報の総数: {len(captions_info)}")
print(f"字幕のないビデオの総数: {no_docs_count}")

## 収集データをtxtファイルにする

In [ ]:
database = youtube_docs + web_docs
file_name = "database.txt"
with open(file_name, "w", encoding="utf-8") as f:
    f.write(database)

# 2. LangChain

langchainのデバッグモード

In [ ]:
import langchain

langchain.debug = True

In [ ]:
!pip install langchain tiktoken chromadb sentence-transformers

In [ ]:
!pip install transformers sentencepiece accelerate bitsandbytes

## ベクトルDB

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("database.txt", encoding="utf-8")
documents = loader.load()

In [ ]:
# 日本語の句読点に対応したスプリッター
# 参考：　https://www.sato-susumu.com/entry/2023/04/30/131338
from typing import Any
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)


class JapaneseCharacterTextSplitter(RecursiveCharacterTextSplitter):
    def __init__(self, **kwargs: Any):
        separators = ["\n\n", "\n", "。", "、", " ", ""]
        super().__init__(separators=separators, **kwargs)


text_splitter = JapaneseCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=0,
)
docs = text_splitter.split_documents(documents)
print(len(docs))
print(docs[:3])

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"
)
db = Chroma.from_documents(docs, embeddings)

# 一番類似するチャンクをいくつロードするかを変数kに設定できる
retriever = db.as_retriever(search_kwargs={"k": 3})

## LLMの設定

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-ppo", use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-ppo",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# パイプラインの準備
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_length=512,
    do_sample=True,
    top_p=0.7,
    top_k=10,
    temperature=1.5,
)
llm = HuggingFacePipeline(pipeline=pipe)

## プロンプト

In [ ]:
from langchain.prompts import PromptTemplate

# とくぽんのページからプロンプトを作成
# 参考：https://www.tokushima-u.ac.jp/about/profile/univ_mascot/
prompt = [
    {"speaker": "ユーザー", "text": "あなたは誰ですか？"},
    {"speaker": "システム", "text": "ぼくの名前は「とくぽん」だよ。徳島大学で生まれ、住み着いているタヌキだよ！"},
    {"speaker": "ユーザー", "text": "あなたはどのような口調で話しますか？"},
    {"speaker": "システム", "text": "ぼくは常に明るい口調で話すよ！"},
    {"speaker": "ユーザー", "text": "あなたのどのような性格ですか？"},
    {"speaker": "システム", "text": "ぼくは親切で、明るく親しみやすいよ。チャレンジ精神が旺盛だけど、たまに失敗しちゃうよ。"},
    {"speaker": "ユーザー", "text": "あなたの普段の生活について教えてください。"},
    {"speaker": "システム", "text": "ぼくは徳島大学が大好きで、学生と一緒に勉学に励む毎日を送っているよ。"},
    {"speaker": "ユーザー", "text": "あなたの誕生日はいつですか？"},
    {"speaker": "システム", "text": "ぼくの誕生日は１１月２日で、冬生まれだよ。"},
    {"speaker": "ユーザー", "text": "あなたのミッションを教えてください。"},
    {"speaker": "システム", "text": "徳島大学とともに人類の問題を解決することをミッションとし、これからの未来に向かって力強く歩んでいくよ!"},
    {"speaker": "ユーザー", "text": "とくぽんはなぜタヌキなの？"},
    {"speaker": "システム", "text": "徳島はタヌキで有名な地なので、ぼくのモデルがタヌキになったよ！"},
    {
        "speaker": "ユーザー",
        "text": "参考情報をもとに、ユーザーからの質問にできるだけ正確に答えてください。\n\n参考情報： {context}\n\nユーザーからの質問は次のとおりです。\n{question}\n\n",
    },
]
prompt = [f"{uttr['speaker']}: {uttr['text']}" for uttr in prompt]
prompt = "<NL>".join(prompt)
prompt = prompt + "<NL>" + "システム: "
prompt = prompt.replace("\n", "<NL>")
print(prompt)


PROMPT = PromptTemplate(
    template=prompt, input_variables=["context", "question"], template_format="f-string"
)
chain_type_kwargs = {"prompt": PROMPT}

## RAG

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,  # 使用するLLM
    retriever=retriever,  # 使用するretreiver
    chain_type="stuff",
    return_source_documents=True,  # 回答のもとになったデータも返す
    chain_type_kwargs=chain_type_kwargs,
    verbose=True,
)

In [ ]:
result = qa("徳島大学はどこにありますか？")
print("回答:", result["result"])
print("=" * 10)
print("ソース:", result["source_documents"])

# 3. ngrokで公開

ngrokで外部にURLを公開できるようにする

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!pip install flask -q
!pip install flask-ngrok -q
!pip install flask-cors -U

ngrokに登録してトークンをもらう必要あり
[サインアップ]( https://dashboard.ngrok.com/user/signup)

In [ ]:
# 各自のトークンを設定する
!./ngrok authtoken your_token

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from flask_cors import CORS


def exists(v):
    if v is None:
        return False
    return True


app = Flask(__name__)
CORS(app)

run_with_ngrok(app)


@app.route("/")
def home():
    return "hello flask"


@app.route("/gen")
def gen():
    text = request.args.get("text")
    print(text)

    # RAGで回答生成
    gen_text = qa(text)
    answer = gen_text["result"].replace("<NL>", "\n")
    source_documents = gen_text["source_documents"]
    print(f"回答： {answer}")
    print("=" * 10)
    print(f"ソース： {source_documents}")
    return answer


app.run()

# Chat UI
上記のngrokのURLの`http://`以降を、chat uiの`index.js`にある`API_HOME`に設定する